In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import itertools

# Generate all interactions

In [2]:
#source https://stackoverflow.com/questions/43106707/generate-all-possible-2-character-combinations-of-a-potential-8-character-string
from itertools import product, combinations

def build(num_chars, length):
    bases = ["U", "C", "A", "G"]
    for letters in product(bases, repeat=num_chars):
        for positions in combinations(range(length), num_chars):
            yield tuple(zip(letters, positions))

In [3]:

pre_mono = list(build(1,8))
post_mono = []
for i in range(len(pre_mono)):
    post_mono.append((pre_mono[i][0],(False,False)))
    
dint_combos = list(build(2,8))
complete_combinations = post_mono+dint_combos

# encode the integer encoding schemes

In [28]:
import csv

mono_integer_dict = {}

#opens csv file, reads into a dictionary
with open('INTEGER_mono_scheme.csv') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        mono_integer_dict[row['base']] = [row['s'],row['t']]


dint_integer_dict = {}

#opens csv file, reads into a dictionary
with open('INTEGER_dint_scheme.csv') as csvfile:
    reader = csv.DictReader(csvfile)
    for row in reader:
        dint_integer_dict[row['dint']] = [row['SS'],row['TT']]

# import eff list to pd.Dataframe

In [32]:
predicted_eff = pd.read_csv("dinucleotide_TIS_eff.csv")

In [33]:
del predicted_eff['lower.bound']
del predicted_eff['upper.bound']

# Encode the sequences according to the scheme

In [34]:
def make_encoding_scheme(num_interactions):
    bases = ['U','C','A','G']
    import itertools

    sequential_diNT_interactions = [''.join(i) for i in itertools.product(bases,repeat=num_interactions)]

    seq_diNT_combos = {}

    for combo in sequential_diNT_interactions:
        seq_diNT_combos[combo] = sequential_diNT_interactions.index(combo)+1

    return seq_diNT_combos

In [35]:
mono_integer_dict
dint_integer_dict

import itertools

def mono_integer_encode(stripped_TIS, encoding_scheme):
    mono_int_encoded = []
    
    for base in stripped_TIS:
        mono_int_encoded.append(encoding_scheme[base])
    return mono_int_encoded
    
def di_integer_encode(stripped_TIS, encoding_scheme, window):
    
    di_int_encoded = []
    
    diNT_interactions = [ "".join(j) for j in zip(*[stripped_TIS[i:] for i in range(window)])]
    
    for interaction in diNT_interactions:
        di_int_encoded.append(encoding_scheme[interaction])
    return di_int_encoded

In [54]:
def flatten_list(l):
    return [item for sublist in l for item in sublist]

In [58]:
integer_encoded = []
for seq in predicted_eff['sequence']:
    stripped_TIS = seq[:6]+seq[9:]

    integer_encoded.append(flatten_list(mono_integer_encode(stripped_TIS,mono_integer_dict) + di_integer_encode(stripped_TIS, dint_integer_dict, 2)))
    

In [65]:
#merge binaries list and efficienies together
to_merge = pd.DataFrame(integer_encoded)

to_merge['efficiency'] = predicted_eff['efficiency']



In [62]:
to_merge

,0,1,2,3,4,5,6,7,8,9,...,21,22,23,24,25,26,27,28,29,efficiency
0,0,1,0,1,1,1,0,0,0,1,...,1,0,1,2,0,3,0,1,1,150
1,1,0,1,1,1,0,0,0,1,0,...,2,2,2,1,1,2,1,3,1,149
2,0,0,1,1,1,0,0,0,1,0,...,2,2,2,1,2,2,2,3,1,148
3,0,0,0,1,1,1,0,0,0,1,...,1,0,1,2,0,3,0,1,1,147
4,0,0,1,1,1,0,0,0,1,0,...,2,2,2,1,1,2,1,3,1,145
5,0,0,1,1,1,0,1,0,1,0,...,2,3,2,1,2,2,2,3,1,144
6,0,0,0,1,1,0,1,0,1,0,...,2,3,2,1,2,2,2,3,1,144
7,0,0,1,1,1,0,0,0,1,0,...,2,2,2,3,2,3,2,3,1,144
8,0,0,0,1,1,0,0,0,1,0,...,2,2,2,1,2,2,2,3,1,143
9,1,0,0,1,1,1,0,0,0,1,...,1,0,1,2,0,3,0,1,1,142


In [66]:
to_merge.to_csv("INTEGER_scheme_for_LM.csv")

# mono only

In [ ]:
binaries_mono = []
for each in predicted_eff['sequence']:
    binaries_mono.append(convert_seq_to_binary(make_keys(each),post_mono))

In [ ]:
binaries_di =[]
for each in predicted_eff['sequence']:
    binaries_di.append(convert_seq_to_binary(make_keys(each), dint_combos))

In [ ]:
mono_df = pd.DataFrame(binaries_mono, columns = post_mono)
mono_df['efficiency'] = predicted_eff['efficiency']
mono_df.to_csv("mono_interactions.csv")

di_df = pd.DataFrame(binaries_di, columns = dint_combos)
di_df['efficiency'] = predicted_eff['efficiency']
di_df.to_csv("di_interactions.csv")

In [ ]:
mono_df

# Make binary vector of seq compared to all combos

In [ ]:
# [first or second tuple] [first or second value in the tuple]
# [0][0] = 1st value in 1st tuple
# [0][1] = 2nd value in 1st tuple
# 
# [1][0] = 1st value in 2nd tuple
# [1][1] = 2nd value in 2nd tuple

# lookup_key(pwm_key)